# Obtención de los datos de asistencia

## Antecedentes

Se requiere obtener los datos de asistencia de unos cantidad mayor de archivos pdf, lo que hace que sea inv
Total de archivos objetivo a analizar 1068.

Del total de archivos objetivos que es posible semi-automatizar su proceso de obtención de los datos es 1047 archivos. Lo es aprox. el 98.03% de los datos.

Actualmente se estima que se tiene cerca del 90% de los colegio

Hay un total de 140 colegios.

In [1]:
from pathlib import Path
import fitz
import os
import pandas as pd
import main
import tabula
import datetime
from constants import *
import numpy as np
import shutil
from Levenshtein import distance
from numba import njit
from joblib import Parallel,delayed
import seaborn as sns
import matplotlib.pyplot as plt
import re
from unicodedata import normalize
from methods import *
from collections import defaultdict


home = Path(Path(os.getcwd()).parent,"Automización Asisitencia")
output_dir = Path(home,"Asistencia SIGE","Output")

## Paso 1 - Digitalización de los Documentos

Creamos un algoritmo que convierte los documentos en un único formato stantard para poder trabajar posteriormente.

Nota: Los algoritmos no son perfectos es bueno hacer la comparación entre el resultado y el documento original.

### Algoritmos de Digitalización Automatica

In [2]:
if not os.path.exists(output_dir):
    os.mkdir(Path(output_dir))

path_paso1 = Path(output_dir,"Paso 1")
if not os.path.exists(path_paso1):
    os.mkdir(Path(path_paso1))


#### Formato 1

Uno de los formatos más importes de automatizar su digitalización, ya que hay un total de 82 colegios que entregaron con este formato. 
El formato de este tipo de documento posee los siguientes datos:
 - RDB 
 - Nombre del establecimiento
 - Mes
 - Curso
 - Tabla de Asistencia por día para cada alumno del curso

In [3]:
dir_format1 = [(path, str(path).split("\\")[-2:]) for path in Path(home,"Asistencia SIGE","Marzo-Nov","Formato 1").glob("**/*.pdf")]

if not os.path.exists(Path(path_paso1,"Formato 1")):
    os.mkdir(Path(path_paso1,"Formato 1"))

error_list = []
    
for dir_pdf, [rbd, name_pdf] in dir_format1:
    doc_pdf = fitz.open(dir_pdf)
    list_blocks = get_info_in_blocks(doc_pdf)
    parse_blocks(list_blocks)
    try:
        df = main.parse_format1(list_blocks)
        if not os.path.exists(Path(path_paso1,"Formato 1",rbd)):
            os.mkdir(Path(path_paso1,"Formato 1",rbd))
        df.to_excel(Path(path_paso1,"Formato 1",rbd,f"{name_pdf[:-4]}.xlsx"))    
    except:
        error_list.append((rbd,name_pdf))
        print("error")
        continue
    
for rbd, name_pdf in error_list:   
    print(f"RDB: {rbd}, Nombre: {name_pdf}\n")

0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2


#### Formato 2

Una variante del formato 1, posee menos información, solo 6 colegios entregaron en este formato.

In [5]:
dir_format2 = [(path, str(path).split("\\")[-2:] ) for path in Path(home,"Asistencia SIGE","Marzo-Nov","Formato 2").glob("**/*.pdf")]

if not os.path.exists(Path(path_paso1,"Formato 2")):
    os.mkdir(Path(path_paso1,"Formato 2"))

error_list = []

for dir_pdf, [rbd ,name_pdf] in dir_format2:
    doc_pdf = fitz.open(dir_pdf)
    list_blocks = get_info_in_blocks(doc_pdf)
    parse_blocks(list_blocks)
    try:
        df = main.parse_format2(list_blocks,int(rbd))
        if not os.path.exists(Path(path_paso1,"Formato 2",rbd)):
            os.mkdir(Path(path_paso1,"Formato 2",rbd))
        df.to_excel(Path(path_paso1,"Formato 2",rbd,f"{name_pdf[:-4]}.xlsx"))        
    except:
        error_list.append((rbd,name_pdf,dir_pdf))
        print("error")
        continue

for rbd, name_pdf, dir_pdf in error_list:   
    print(f"RDB: {rbd}, Nombre: {name_pdf}, url: {dir_pdf}\n")

3
error
3
error
3
error
3
error
3
4
5
6
3
error
3
error
3
error
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
error
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
3
4
5
6
RDB: 10087, Nombre: SIGE asistencia abril 4to.pdf, url: c:\Users\Sebastián Carrasco\Desktop\Practica CIAE\Automización Asisitencia\Asistencia SIGE\Marzo-Nov\Formato 2\10087\SIGE asistencia abril 4to.pdf

RDB: 10087, Nombre: SIGE asistencia agosto 4to.pdf, url: c:\Users\Sebastián Carrasco\Desktop\Practica CIAE\Automización Asisitencia\Asistencia SIGE\Marzo-Nov\Formato 2\10087\SIGE asistencia agosto 4to.pdf

RDB: 10087, Nombre: SIGE asistencia julio 4to.pdf, url: c:\Users\Sebastián Carrasco\Desktop\Practica CIAE\Automización Asisitencia

#### Formato 3

Variante del formato 1, este documento solo posee la tabla de asistencia diaria por cada alumno del curso, solo 3 colegios enviaron con este formato

In [ ]:
dir_format3 = [(path, str(path).split("\\")[-2:]) for path in Path(home,"Asistencia SIGE","Marzo-Nov","Formato 3").glob("**/*.pdf")]

if not os.path.exists(Path(path_paso1,"Formato 3")):
    os.mkdir(Path(path_paso1,"Formato 3"))

error_list = []

for dir_pdf, [rbd ,name_pdf] in dir_format3:
    doc_pdf = fitz.open(dir_pdf)
    list_blocks = get_info_in_blocks(doc_pdf)
    parse_blocks(list_blocks)
    try:
        df = main.parse_format3(doc_pdf,int(rbd),name_pdf[:-4])     
        if not os.path.exists(Path(path_paso1,"Formato 3",rbd)):
            os.mkdir(Path(path_paso1,"Formato 3",rbd))
        df.to_excel(Path(path_paso1,"Formato 3",rbd,f"{name_pdf[:-4]}.xlsx"))               
    except:
        error_list.append((rbd,name_pdf,dir_pdf))
        print("error")
        continue

for rbd, name_pdf, dir_pdf in error_list:   
    print(f"RDB: {rbd}, Nombre: {name_pdf}, url: {dir_pdf}\n")

#### Formato Cuadernos Digitales

In [ ]:
dir_format_p = [(path, str(path).split("\\")[-2:]) for path in Path(home,"Asistencia SIGE","Marzo-Nov","Formato Cuaderno Digital").glob("**/*.pdf")]

if not os.path.exists(Path(path_paso1,"Formato Digital")):
    os.mkdir(Path(path_paso1,"Formato Digital"))

error_list = []

for dir_pdf, [rbd, name_pdf] in dir_format_p:
    list_df = tabula.read_pdf(dir_pdf,pages="all")
    try:
        for j, df in enumerate(list_df):
            mes, _ = df.columns[0].split(" ")
            month, days = PARSE_STR_MONTH_TO_DATA[mes]
            dates = [datetime.date(2022, month, i).isoformat() for i in range(1, days+1)]
            n = len(df.columns)-(len(dates)+1)
            df = df.drop(columns=df.columns[-n:]).reset_index(drop=True)
            df.columns = ["Alumno"] + dates
            df = df.replace({"|":-1,
                             "●":1,
                             "X":0,
                             "-":-3,
                            })
            df = df.assign(RBD = rbd)
            try:
                df.to_excel(Path(path_paso1,"Formato Digital",rbd,f"{name_pdf[:-4]}_{j}.xlsx"),index=False)
            except:
                os.mkdir(Path(path_paso1,"Formato Digital",rbd))
                df.to_excel(Path(path_paso1,"Formato Digital",rbd,f"{name_pdf[:-4]}_{j}.xlsx"),index=False)
                
    except:
        error_list.append((rbd,name_pdf))
        continue
    
for rbd, name_pdf in error_list:   
    print(f"RDB: {rbd}, Nombre: {name_pdf}\n")

### Tabla de Ejemplo

In [ ]:
df = pd.read_excel(Path(path_paso1,"Formato 1","8521","abril.xlsx"))
print(f"Las columnas del archivo\n {df.columns}\n")
df.head(5)

Las columnas del archivo
 Index(['Alumno', '2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04',
       '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-09',
       '2022-04-10', '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14',
       '2022-04-15', '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19',
       '2022-04-20', '2022-04-21', '2022-04-22', '2022-04-23', '2022-04-24',
       '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29',
       '2022-04-30', 'RBD'],
      dtype='object')



,Alumno,2022-04-01,2022-04-02,2022-04-03,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08,2022-04-09,...,2022-04-22,2022-04-23,2022-04-24,2022-04-25,2022-04-26,2022-04-27,2022-04-28,2022-04-29,2022-04-30,RBD
0,Andrade Gomez Carla,1,-1,-1,1,1,1,1,1,-1,...,1,-1,-1,1,1,1,1,0,-1,8521
1,Arancibia Jara Loreto,1,-1,-1,1,1,0,0,1,-1,...,1,-1,-1,1,1,1,1,1,-1,8521
2,Bernaes Jimenez Pedro,1,-1,-1,1,1,1,1,1,-1,...,1,-1,-1,1,1,1,1,1,-1,8521
3,Bustos Paredes Violeta,1,-1,-1,1,1,1,1,1,-1,...,1,-1,-1,1,1,1,1,1,-1,8521
4,Cabos Mateluna Mia,1,-1,-1,1,1,1,1,1,-1,...,1,-1,-1,1,1,1,1,1,-1,8521


Asistencia: 1
Inasistencia: 0
No Trabajable: -1
Otros: -3

In [ ]:
del df

## Paso 2 - Revisión de Digitalización

Arreglar el formato de las otras tablas para ser unificadas más adelante

In [221]:
path_paso2 = Path(output_dir,"Paso 2")

if not os.path.exists(path_paso2):
    os.mkdir(path_paso2)

## Paso 3 - Creación Tabla Única

Unir las tablas en una sola tabla excel

In [549]:
path_paso3 = Path(output_dir, "Paso 3")

if not os.path.exists(path_paso3):
    os.mkdir(path_paso3)

if os.path.exists(Path(path_paso3, "merge.xlsx")):
    os.remove(Path(path_paso3, "merge.xlsx"))


In [550]:
class TablaAsistencia:
    def __init__(self, df: pd.DataFrame, rbd: int):
        self.Tabla: pd.DataFrame = df
        self.RBD: int = rbd

    def get_table(self) -> pd.DataFrame:
        self.Tabla.columns = pd.to_datetime(self.Tabla.columns).map(
            lambda x: x.strftime("%m-%d")).astype("string")
        return self.Tabla.assign(RBD=self.RBD)

def create_table(path_excel:Path)->TablaAsistencia:
    dtypes = defaultdict(lambda: "int8")
    dtypes["Alumno"] = "string"
    dtypes["RBD"] = "uint32"
    
    df = pd.read_excel(path_excel, index_col="Alumno", dtype=dtypes)
    
    return TablaAsistencia(df.drop(columns="RBD"),df.RBD[0])

In [551]:
dir_aux: list[Path] = [path for path in path_paso2.glob("**/*.xlsx")]
list_df: list[pd.DataFrame] = []
for excel in dir_aux:
    try:
        list_df.append(create_table(excel).get_table())
    except:
        print(excel)


In [552]:
#create_table("c:/Users/Sebastián Carrasco/Desktop/Practica CIAE/Automización Asisitencia/Asistencia SIGE/Output/Paso 2/Excel/9150/Mayo.xlsx")

In [553]:
for i, df in enumerate(list_df):
    try:
        df.drop(columns="RBD").stack().reset_index().rename(columns={"level_1":"Fecha",0:"Asistencia"})
    except:
        print(dir_aux[i])

In [554]:
def transform_vertical(x: pd.DataFrame)->pd.DataFrame:
    # Obtenemos el rbd del establecimiento
    rbd = x["RBD"].unique()[0]
    # Compactamos el DataFrame sin el código RBD
    x = x.drop(columns="RBD").stack().reset_index().rename(columns={"level_1":"Fecha",0:"Asistencia"})
    # Retornamos el DataFrame compactado con aplicando el RBD en toda la columna
    return x.assign(RBD = rbd)

Aplicamos la función a cada DataFrame de la lista.

In [555]:
#list_df = list(map(lambda x: transform_vertical(x), list_df))
list_error = []
for i, df in enumerate(list_df):
    try:
        list_df[i] = transform_vertical(df)
    except:
        list_error.append((dir_aux[i]))
print(f"Cantidad de Errores: {len(list_error)}")

Cantidad de Errores: 0


In [556]:
list_error

[]

Concadenamos todos los DataFrame

In [557]:
df_merge = pd.concat(list_df,axis=0, ignore_index=True)
df_merge

,Alumno,Fecha,Asistencia,RBD
0,1 Acevedo Quiroz Valentina Florencia,04-01,1,8514
1,1 Acevedo Quiroz Valentina Florencia,04-02,-1,8514
2,1 Acevedo Quiroz Valentina Florencia,04-03,-1,8514
3,1 Acevedo Quiroz Valentina Florencia,04-04,1,8514
4,1 Acevedo Quiroz Valentina Florencia,04-05,0,8514
...,...,...,...,...
769344,42 Vásquez Morales Jean Philippe,09-26,1,9510
769345,42 Vásquez Morales Jean Philippe,09-27,1,9510
769346,42 Vásquez Morales Jean Philippe,09-28,1,9510
769347,42 Vásquez Morales Jean Philippe,09-29,0,9510


In [558]:
list(df_merge[df_merge.duplicated()].Fecha.unique())

['05-01']

In [559]:
#df_merge[(df_merge.RBD==9150) & (df_merge.Fecha == '06-28')].sort_values(["Alumno"])

In [560]:
df_merge[df_merge.duplicated()].RBD.unique()

array([9307], dtype=uint32)

In [561]:
#list(df_merge[df_merge.RBD==9150].sort_values("Fecha")["Fecha"].unique())

In [562]:
#df_merge[df_merge.RBD==9150].groupby("Alumno").size()

In [563]:
df_merge[df_merge.Alumno.map(lambda x: type(x)==int or type(x)==float)]

,Alumno,Fecha,Asistencia,RBD


In [564]:
def quitar_indice_nombre(name:str)->str:
    list_str = name.split()
    if list_str[0].isnumeric():
        return " ".join(list_str[1:])
    else:
        return " ".join(list_str)


df_merge.Alumno = df_merge.Alumno.map(lambda x: quitar_indice_nombre(x))

In [565]:
df_merge = df_merge.dropna(axis=0)

In [566]:
df_merge["Alumno"] = df_merge["Alumno"].map(lambda x: x.title())
df_merge = df_merge.astype({"Alumno": "string",
                            "Fecha": "string",
                            })
df_merge["Asistencia"] = pd.to_numeric(df_merge["Asistencia"],downcast="integer")
df_merge["RBD"] = pd.to_numeric(df_merge["RBD"].map(lambda x: int(x)),downcast="unsigned")
df_merge

,Alumno,Fecha,Asistencia,RBD
0,Acevedo Quiroz Valentina Florencia,04-01,1,8514
1,Acevedo Quiroz Valentina Florencia,04-02,-1,8514
2,Acevedo Quiroz Valentina Florencia,04-03,-1,8514
3,Acevedo Quiroz Valentina Florencia,04-04,1,8514
4,Acevedo Quiroz Valentina Florencia,04-05,0,8514
...,...,...,...,...
769344,Vásquez Morales Jean Philippe,09-26,1,9510
769345,Vásquez Morales Jean Philippe,09-27,1,9510
769346,Vásquez Morales Jean Philippe,09-28,1,9510
769347,Vásquez Morales Jean Philippe,09-29,0,9510


In [567]:
df_merge = df_merge.drop_duplicates().reset_index(drop=True)

In [568]:
df_merge[df_merge.duplicated()].sort_values(by="Fecha")

,Alumno,Fecha,Asistencia,RBD


In [569]:
df_merge[df_merge["Alumno"].isna()]["RBD"].unique()

array([], dtype=uint16)

In [570]:
df_merge[df_merge["Asistencia"].isna()]["RBD"].unique()

array([], dtype=uint16)

In [571]:
len(df_merge.RBD.unique())

88

In [572]:
df_merge = df_merge.pivot(index=["Alumno","RBD"],
                          columns="Fecha",
                          values="Asistencia").reset_index(level=[0,1],
                                                           col_level=1).fillna(-5)

In [573]:
# Guardamos el documento unido        
df_merge.to_csv(Path(path_paso3,"merge.csv"),encoding="utf-16",index=False)

Revisamos si la tabla que se creo es identica a la que obtuvimos

In [574]:
from collections import defaultdict
dtypes = defaultdict(lambda: "int8")
dtypes["Alumno"] = "string"
dtypes["RBD"] = "uint32"

df = pd.read_csv(Path(path_paso3, "merge.csv"),
                 encoding="utf-16", dtype=dtypes)
df.info()
# if df.equals(df_merge):
#    #del df_merge
#    print("Son iguales! Todo salió bien.")
# else:
#    print("Algo salió mal, se sugiere revisar los parámetros de la conversión a csv.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Columns: 308 entries, Alumno to 12-31
dtypes: int8(306), string(1), uint32(1)
memory usage: 954.4 KB


In [575]:
df

,Alumno,RBD,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
0,Abarca Barraza Benjamín,25613,-1,-1,-1,-1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,Abarca Birocchi Marcelo Sebastián,25171,-3,1,1,1,-1,-1,1,1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Abarca Díaz Dominique,9993,-1,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3,Abarca Gómez Maickol,10183,-3,-3,-3,-3,-3,-3,-3,-3,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
4,Abarca Vásquez Francisca,31337,-1,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,Ávila Toledo Antonella,25061,-1,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3069,Ñanco Palomo Felipe Andrés,10250,-1,-1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3070,Ñancufil Arévalo Abner,9551,-1,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3071,Órdenes Calvert Emilia,25613,-1,-1,-1,-1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


In [576]:
len(df["RBD"].unique())

88

In [577]:
# Quita los números antes del nombre (residuos de los indices SIGE)

df.Alumno[df.Alumno.map(lambda x: x.split()[0].isnumeric())] = df.Alumno.map(lambda x: " ".join(x.split()[1:]))

In [578]:
aux = df.groupby(by="Alumno").size()
aux[aux>1]

Alumno
Escanilla Toledo Angelo      2
Hernández Ibarra Agustina    2
Joseph                       2
Zúñiga Cordero Irenne        2
dtype: int64

In [579]:
list_rep = ["Escanilla Toledo Angelo",
            "Hernández Ibarra Agustina",
            "Zúñiga Cordero Irenne",
            ]

In [580]:
df[df.Alumno.map(lambda x: x in list_rep)]

,Alumno,RBD,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
832,Escanilla Toledo Angelo,10010,-5,-5,-5,-5,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
833,Escanilla Toledo Angelo,10251,-5,-5,-5,-5,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1242,Hernández Ibarra Agustina,9758,-1,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1243,Hernández Ibarra Agustina,10619,-5,-5,-5,-5,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3049,Zúñiga Cordero Irenne,10302,-3,-3,-3,-3,-3,-3,0,0,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
3050,Zúñiga Cordero Irenne,25803,-5,-5,-5,-5,-5,-5,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


Se puede concluir que, dado el nombre es el mismo, pero los RBD distintos, hay niños que se cambiaron de colegio durante el semestre.

Recordemos que solo nos interesa desde Marzo hasta Noviembre, por lo que hay que filtrar datos.

In [581]:
df = df[df.columns[df.columns.map(lambda x: True if len(x.split("-"))==1 else int(x.split("-")[0])<12)]]
df

,Alumno,RBD,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,...,11-21,11-22,11-23,11-24,11-25,11-26,11-27,11-28,11-29,11-30
0,Abarca Barraza Benjamín,25613,-1,-1,-1,-1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
1,Abarca Birocchi Marcelo Sebastián,25171,-3,1,1,1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
2,Abarca Díaz Dominique,9993,-1,1,1,1,-1,-1,1,1,...,0,1,1,1,1,-1,-1,1,1,1
3,Abarca Gómez Maickol,10183,-3,-3,-3,-3,-3,-3,-3,-3,...,0,0,1,1,0,-1,-1,1,1,1
4,Abarca Vásquez Francisca,31337,-1,1,1,1,-1,-1,1,1,...,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3068,Ávila Toledo Antonella,25061,-1,1,1,1,-1,-1,1,1,...,1,1,1,1,1,-1,-1,1,1,1
3069,Ñanco Palomo Felipe Andrés,10250,-1,-1,1,1,-1,-1,1,1,...,1,1,1,1,1,-1,-1,1,1,1
3070,Ñancufil Arévalo Abner,9551,-1,1,1,1,-1,-1,1,1,...,1,1,1,1,1,-1,-1,1,1,1
3071,Órdenes Calvert Emilia,25613,-1,-1,-1,-1,-1,-1,1,1,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5


## Paso 4 - Juntar información de otras tablas

Unir tablas de asistencia con las tablas de alumnos e IVE

### Tabla de Alumnos

#### Estandarización de los datos de los Alumnos

In [582]:
df_alumnos = pd.read_excel(Path(home, "Datos No Modificar", "alumnos.xlsx"),
                           sheet_name="alumnos",
                           dtype={"institucion_id":     "uint16",
                                  "rbd":          "uint16",
                                  "nombre_inst":  "string",
                                  "curso_id":  "uint32",
                                  "nivel_crs":   "uint8",
                                  "nombre_crs": "category",
                                  "usuario_id":  "uint32",
                                  "nombre":  "string",
                                  "apellido_paterno":  "string",
                                  "apellido_materno":  "string",
                                  "genero": "category",
                                  "rut":  "string",
                                  "fecha_nacimiento":  "string",
                                  })


observamos el dataset

In [583]:
df_alumnos.head(5)

,institucion_id,rbd,nombre_inst,curso_id,nivel_crs,nombre_crs,usuario_id,nombre,apellido_paterno,apellido_materno,genero,rut,fecha_nacimiento
0,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773965,ANTONELLA IGNACIA,Retamales,MUÑOZ,F,24220965-6,2013-03-16
1,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,969647,ISIDORA AGUSTINA,Fuentes,CERDA,F,23641968-1,2011-05-12
2,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773944,AMARO ALEXANDER,Arraño,ANGLA,M,23811691-0,2011-11-28
3,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773945,MARTÍN ELÍAS,Guerra,ZÚÑIGA,M,23864538-7,2012-02-07
4,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126189,VICENTE IGNACIO,Fernández,RETAMAL,M,23944621-3,2012-04-30


Vemos que hay problemas con el texto, están en diferentes formatos, algunos solo en mayúsculas y otros solo tienen la primera letra en mayúscula.

In [584]:
df_alumnos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4778 entries, 0 to 4777
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   institucion_id    4778 non-null   uint16  
 1   rbd               4778 non-null   uint16  
 2   nombre_inst       4778 non-null   string  
 3   curso_id          4778 non-null   uint32  
 4   nivel_crs         4778 non-null   uint8   
 5   nombre_crs        4778 non-null   category
 6   usuario_id        4778 non-null   uint32  
 7   nombre            4778 non-null   string  
 8   apellido_paterno  4777 non-null   string  
 9   apellido_materno  4643 non-null   string  
 10  genero            4778 non-null   category
 11  rut               4584 non-null   string  
 12  fecha_nacimiento  4491 non-null   string  
dtypes: category(2), string(6), uint16(2), uint32(2), uint8(1)
memory usage: 294.3 KB


El documento posee NaNs dado que algunos alumnos no tienen registrado un RUT y/o fecha de nacimiento. También se da el caso, que hay alumnos que tienen nombre y apellido paterno o solo nombre. Entonces, para resolver este problema, estos datos los pasaremos a string y los NaNs los reemplazaremos por strings vacios ("") para denotar que no hay registro y minimizar el espacio y el uso de memoria del dataset.

In [585]:
df_alumnos = df_alumnos.replace(np.nan,"")

Verificamos el resultado

In [586]:
df_alumnos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4778 entries, 0 to 4777
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   institucion_id    4778 non-null   uint16  
 1   rbd               4778 non-null   uint16  
 2   nombre_inst       4778 non-null   string  
 3   curso_id          4778 non-null   uint32  
 4   nivel_crs         4778 non-null   uint8   
 5   nombre_crs        4778 non-null   category
 6   usuario_id        4778 non-null   uint32  
 7   nombre            4778 non-null   string  
 8   apellido_paterno  4778 non-null   string  
 9   apellido_materno  4778 non-null   string  
 10  genero            4778 non-null   category
 11  rut               4778 non-null   string  
 12  fecha_nacimiento  4778 non-null   string  
dtypes: category(2), string(6), uint16(2), uint32(2), uint8(1)
memory usage: 294.4 KB


Ahora, estandarizaremos los nombres y apellidos de los alumnos.

In [587]:
for feature in ["nombre","apellido_paterno","apellido_materno"]:
    df_alumnos[feature] = list(map(lambda x: x.lower().title() ,df_alumnos[feature].to_list()))
df_alumnos.head(15)

,institucion_id,rbd,nombre_inst,curso_id,nivel_crs,nombre_crs,usuario_id,nombre,apellido_paterno,apellido_materno,genero,rut,fecha_nacimiento
0,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773965,Antonella Ignacia,Retamales,Muñoz,F,24220965-6,2013-03-16
1,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,969647,Isidora Agustina,Fuentes,Cerda,F,23641968-1,2011-05-12
2,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773944,Amaro Alexander,Arraño,Angla,M,23811691-0,2011-11-28
3,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,773945,Martín Elías,Guerra,Zúñiga,M,23864538-7,2012-02-07
4,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126189,Vicente Ignacio,Fernández,Retamal,M,23944621-3,2012-04-30
5,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126199,Joaquín Ignacio,Llanos,Mora,M,23945298-1,2012-05-08
6,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126185,Agustín Isaac,Molina,Silva,M,23956124-1,2012-05-23
7,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126178,Noemí Ignacia,Aguayo,Verdejo,F,23976729-k,2012-06-10
8,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126187,Martín Marcelo,Burgos,Ortiz,M,24115492-0,2012-11-12
9,4049,10622,ESCUELA PARTICULAR MARVIT SCHOOL,215545,4,A,126177,Pascal Isidora,Bolbarán,Zúñiga,F,24120339-5,2012-11-17


In [588]:
aux = df_alumnos.groupby(by="usuario_id").size()
df_alumnos[df_alumnos.usuario_id.map(lambda x: x in aux[aux>1].index)].sort_values("usuario_id")

,institucion_id,rbd,nombre_inst,curso_id,nivel_crs,nombre_crs,usuario_id,nombre,apellido_paterno,apellido_materno,genero,rut,fecha_nacimiento
2649,9514,10204,ESCUELA CAPITAN JOSE LUIS ARANEDA,214125,4,A,2152986,Dorian,Pastenes,Pastenes,M,24060042-0,
4776,9514,10204,ESCUELA CAPITAN JOSE LUIS ARANEDA,214125,4,A,2152986,Dorian,Pastenes,Pastenes,M,24060042-0,
2650,9514,10204,ESCUELA CAPITAN JOSE LUIS ARANEDA,214125,4,A,2152987,Nilk Dixon,Suarez,Barrueto,M,23993265-7,
4775,9514,10204,ESCUELA CAPITAN JOSE LUIS ARANEDA,214125,4,A,2152987,Nilk Dixon,Suarez,Barrueto,M,23993265-7,
2680,3724,9738,COLEGIO PARQUE LAS AMERICAS,212766,4,A,2154438,Alexander,Agamez,,M,100499504-6,2012-06-01
4777,3724,9738,COLEGIO PARQUE LAS AMERICAS,212766,4,A,2154438,Alexander,Agamez,,M,100499504-6,
2652,3890,10183,ESCUELA PARTICULAR CARLOS PEZOA VELIZ,214051,4,A,2154471,Victoria,Simancas,Villalobos,F,100560354-0,
4774,3890,10183,ESCUELA PARTICULAR CARLOS PEZOA VELIZ,214051,4,A,2154471,Victoria,Simancas,Villalobos,F,100560354-0,


Eliminamos los datos repetidos y conservamos la mayor cantidad de información de los alumnos

In [589]:
list_index_rep = [4777]

df_alumnos = df_alumnos.drop(index=list_index_rep)

In [590]:
df_alumnos = df_alumnos.drop_duplicates()

In [591]:
df_alumnos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4774 entries, 0 to 4773
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   institucion_id    4774 non-null   uint16  
 1   rbd               4774 non-null   uint16  
 2   nombre_inst       4774 non-null   string  
 3   curso_id          4774 non-null   uint32  
 4   nivel_crs         4774 non-null   uint8   
 5   nombre_crs        4774 non-null   category
 6   usuario_id        4774 non-null   uint32  
 7   nombre            4774 non-null   object  
 8   apellido_paterno  4774 non-null   object  
 9   apellido_materno  4774 non-null   object  
 10  genero            4774 non-null   category
 11  rut               4774 non-null   string  
 12  fecha_nacimiento  4774 non-null   string  
dtypes: category(2), object(3), string(3), uint16(2), uint32(2), uint8(1)
memory usage: 331.3+ KB


In [592]:
aux = df_alumnos.groupby(by="usuario_id").size()
df_alumnos[df_alumnos.usuario_id.map(lambda x: x in aux[aux>1].index)].sort_values("usuario_id")

,institucion_id,rbd,nombre_inst,curso_id,nivel_crs,nombre_crs,usuario_id,nombre,apellido_paterno,apellido_materno,genero,rut,fecha_nacimiento


No hay más datos duplicados.

Ahora que el dataset esta estandarizado, podemos unirlo con el dataset de asistencias.

### Obteniendo el usuario_id de la tabla de Alumnos a partir del nombre de la tabla de Asistencias

[Implementación de la distancia de Levenshtein en Python](https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python)

In [593]:
@njit
def LD(s,t):
    s = ' ' + s
    t = ' ' + t
    d = {}
    S = len(s)
    T = len(t)
    for i in range(S):
        d[i, 0] = i
    for j in range (T):
        d[0, j] = j
    for j in range(1,T):
        for i in range(1,S):
            if s[i] == t[j]:
                d[i, j] = d[i-1, j-1]
            else:
                d[i, j] = min(d[i-1, j], d[i, j-1], d[i-1, j-1]) + 1
    return d[S-1, T-1]

In [594]:
def contador(list1:list,list2:list)->float:
    c = 0
    i = 0
    n = len(list1)
    while i<len(list1):
        j=0
        while j<len(list2):
            if list1[i] == list2[j]:
                c+=1
                list1.pop(i)
                list2.pop(j)
                break
            else:
                j+=1
        else:
            i+=1
    else:
        return c/n

In [595]:
def combination(matrix:list,i:int,idx:list,a:list):
    if i<len(matrix):
        for j in range(len(matrix[i])):
            if j not in idx :
                yield from combination(matrix,i+1,idx+[j],a+[matrix[i][j]])
    else:
        #Guardo en una tupla los valores de la matriz y sus indices
        yield a

def create_matrix(list_A:list, list_B:list, fun):
    return np.array(list(map(lambda B: np.array(list(map(lambda A: fun(A, B),
                                                         list_A))),
                             list_B)))
    
def string_equals(str1:str,str2:str)->int:
    return int(str1 == str2)

def string_in_other_string(str1:str,str2:str)->float:
    list1 = str1.split().copy()
    list2 = str2.split().copy()
    list3 = str1.split().copy()
    list4 = str2.split().copy()

    return max([contador(list1,list2), contador(list4,list3)])

def comb_min_distance(name_a:str,name_b:str)->float:
    
    matrix = create_matrix(name_a.split(), name_b.split(), LD)
    # Return minimum distance between words
    if matrix.shape[0]>matrix.shape[1]:
        matrix = matrix.T
    return min(list(map(lambda x:sum(x),list(combination(matrix,0,[],[])))))

def resolve_matrix_equals(matrix:np.array, list_asist:list, list_idx:list)->list:
    output = []
    
    i=0
    while i < len(matrix):
        opt = matrix[i].max()
        idx = matrix[i].argmax()
        if np.sum(matrix[i]>=opt)==1 and np.sum(matrix[:,idx]>=opt)==1 and opt == 1:
            output.append((list_asist.pop(idx),list_idx.pop(i)))
            
            
            matrix = np.delete(np.delete(matrix,idx,1),i,0)
        else:
            i+=1        
    return [output,list_asist,list_idx]

def resolve_matrix_LD(matrix:np.array, list_asist:list, list_idx:list, th:float)->list:
    output = []
    
    i=0
    while i < len(matrix):
        if matrix[i].size > 0:
            opt = matrix[i].min()
            idx = matrix[i].argmin()
            if np.sum(matrix[i]<=opt)==1 and np.sum(matrix[:,idx]<=opt)==1 and opt<th:
                output.append((list_asist.pop(idx),list_idx.pop(i)))
                matrix = np.delete(np.delete(matrix,idx,1),i,0)
            else:
                i+=1
        else:
            i+=1   
    return [output,list_asist,list_idx]

[Otsu's  Method Implementación en Python](https://en.wikipedia.org/wiki/Otsu%27s_method#Python_implementation])

In [596]:
def compute_otsu_criteria(im, th):
    # create the threshold image
    threshold_im = np.zeros(im.shape)
    threshold_im[im >= th] = 1

    # compute weights
    nb_pixels = im.size
    nb_pixels1 = np.count_nonzero(threshold_im)
    weight1 = nb_pixels1 / nb_pixels
    weight0 = 1 - weight1

    # if one the classes is empty, eg all pixels are below or above the threshold, that threshold will not be considered
    # in the search for the best threshold
    if weight1 == 0 or weight0 == 0:
        return np.inf

    # find all pixels belonging to each class
    val_pixels1 = im[threshold_im == 1]
    val_pixels0 = im[threshold_im == 0]

    # compute variance of these classes
    var0 = np.var(val_pixels0) if len(val_pixels0) > 0 else 0
    var1 = np.var(val_pixels1) if len(val_pixels1) > 0 else 0

    return weight0 * var0 + weight1 * var1

def best_threshold_otsu_method(listA:list, listB:list)->int:
    im = create_matrix(listA,listB,comb_min_distance)
    threshold_range = range(np.max(im)+1)
    criterias = [compute_otsu_criteria(im, th) for th in threshold_range]
    return threshold_range[np.argmin(criterias)]

In [597]:
def get_alumno_id(df_asistencia: pd.DataFrame, df_alumnos: pd.DataFrame) -> pd.DataFrame:

    # Filtramos y ordenamos los datos que necesitamos de los datasets
    # para su posterior
    df_asistencia_f, df_alumnos_f = filtrado_df(df_asistencia, df_alumnos)
    
    # Calculamos un threshold para cuando queramos clasificar por la distancia
    # de Levenshtein
    
    best_threshold = np.array([best_threshold_otsu_method(df_asistencia_f["name"][df_asistencia_f["RBD"]==rbd].to_list(),
                                                          df_alumnos_f["name"][df_alumnos_f["rbd"]==rbd].to_list()) for rbd in df_asistencia_f.RBD.unique()])
    th_mean = best_threshold.mean()
    th_std = best_threshold.std(ddof=1)
    
    # Obtenemos el conjunto de todos los RBD del DataFrame de asistencia
    rbds = df_asistencia_f.RBD.unique()

    # Realizamos, de manera paralela, la búsqueda de los ids dado un nombre por RBD,
    # la guardaremos todos los resultados en una lista
    resultados = [get_id_for_rbd(df_asistencia_f, df_alumnos_f, rbd, np.ceil(th_mean))
                for rbd in rbds]

    # Dado que los resultados están todos en listas, unimos todas las lista en una sola.
    #
    # La lista resultante tendrá pares (nombre, id) que es lo que necesitamos para
    # identificar al alumno en el dataset de asistencias.
    #
    list_id_nombre = []
    res_id_nombre = []
    list_names_asist = []
    list_alumnos_idx = []
    
    for res_rbd in resultados:
        id_nombre, names_asist, alumnos_idx = res_rbd
        res_id_nombre += id_nombre
        list_names_asist += names_asist
        list_alumnos_idx += alumnos_idx
    
    for res in res_id_nombre:
            list_id_nombre += res
    
    # Ahora que se tienen la lista de los pares (nombre,id) podemos separarlos
    # ordenadamente, de manera que no perder nuestro trabajo.
    lista_nombre, lista_id = zip(*list_id_nombre)
    #
    # Lo guardamos en un nuevo DataFrame
    df_name_id = pd.DataFrame({"name_asist": lista_nombre,
                              "name_id": lista_id
                              })
    #
    # y retornamos el dataset de la asistencia asignándole a cada alumno su respectiva id
    #
    #df_aux = df_name_id.merge(right=df_asistencia_f[["name","Alumno"]],left_on="name_asist",right_on="name",how="outer")
    #df_aux = df_aux.merge(right=df_alumnos_f[["name","usuario_id"]],left_on="name_id",right_on="name",how="outer")
    #return df_asistencia.merge(right=df_aux[["Alumno","usuario_id"]],on="Alumno").dropna()
    df_output = df_name_id.merge(right=df_asistencia_f,left_on="name_asist",right_on="name")
    df_output = df_output.merge(right=df_alumnos_f,left_on="name_id",right_on="name")
    ## return
    print("Nombre No Incluidos del Dataset Asistencia\n")
    list_names_asist.sort()
    for name in list_names_asist:
        print(f"{name}")
    print(f"\nTotal no incluidos: {len(list_names_asist)}\n")
        
    print("\nNombre No Incluidos del Dataset Idx\n")
    list_alumnos_idx.sort()
    for name in list_alumnos_idx:
        print(f"{name}")
    print(f"\nTotal no incluidos: {len(list_alumnos_idx)}\n")
    
    # Filtrado
    df_output =  df_output[df_output.RBD == df_output.rbd].drop_duplicates()
    # Merge
    return df_output


def filtrado_df(df_asistencia: pd.DataFrame, df_alumnos: pd.DataFrame) -> list:

    # Filtramos el dataset de asistencias con lo que se necesita
    df_asistencia_f = df_asistencia[["RBD", "Alumno"]].sort_values(by="Alumno")

    # Filtramos y arreglamos el dataset de los ids para poder trabajar
    # eficientemente con los nombres.

    # Arreglamos los nombres y apellidos de manera que tenga el mismo formato
    # que los nombres de los alumnos del otro dataset
    df_alumnos_f = df_alumnos.assign(Alumno=lambda x: (
        x.apellido_paterno + " " + x.apellido_materno + " " + x.nombre))

    # Filtramos los datos que necesitamos
    df_alumnos_f = df_alumnos_f[["usuario_id", "Alumno", "rbd"]].sort_values(by="Alumno")

    # Normalizamos el string de los nombres de los alumnos en cada dataset
    df_asistencia_f = df_asistencia_f.assign(
        name=lambda x: x.Alumno.map(lambda y: normalizar_strings(y.lower())))
    df_alumnos_f = df_alumnos_f.assign(
        name=lambda x: x.Alumno.map(lambda y: normalizar_strings(y.lower())))
    # Retornamos los dataset filtrados y arreglados para trabajar
    return [df_asistencia_f, df_alumnos_f]


def normalizar_string(s: str) -> str:

    # -> NFD y eliminar diacríticos
    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
               normalize("NFD", s), 0, re.I
               )

    # -> NFC
    return normalize('NFC', s)

def normalizar_strings(s: str) -> str:
    list_str = s.split()
    for char in "/?¿!|°*+_{},.&%#@°;":
        list_str = list(map(lambda x: x.replace(char,""),list_str))
    
    list_str = list(map(lambda x: x.strip(),list_str))
    
    list_str = list(map(lambda x: normalizar_string(x),list_str))
    return " ".join(list_str)


def get_id_for_rbd(df_asistencia_f, df_alumnos_f, rbd, th):

    # Primero filtraremos los nombres e ids de los datasets por el RBD dado
    # Así obtendremos eficientemente la información necesaria para aplicar
    # nuestro algoritmo de NLP para identificar el id dl alumno en base a
    # su nombre.
    #
    # El resultado de los datos filtrados en cada dataset serán guardados en listas
    list_names_asist = df_asistencia_f["name"][df_asistencia_f["RBD"] == rbd].to_list()
    
    list_alumnos_idx = df_alumnos_f["name"][df_alumnos_f["rbd"] == rbd].to_list()

    # Con ambas listas de nombres obtenidas podemos aplicar una función que se encargue
    # de entregar un listado de pares (nombre_alumno, id)

    return get_id(list_names_asist, list_alumnos_idx, th)


def get_id(list_names_asist: list, list_alumnos_idx: list, th:float) -> list:

    # Para obtener el id pasaremos los nombres por una serie de algoritmos estratificados
    
    list_asist_idx = []
    
    
    for fun_in_matrix in [string_equals,string_in_other_string]:
        
        if len(list_names_asist)==0 or len(list_alumnos_idx)==0: 
            break
        else:
            matrix = create_matrix(list_names_asist, list_alumnos_idx, fun_in_matrix)

            output, list_names_asist, list_alumnos_idx = resolve_matrix_equals(matrix,list_names_asist,list_alumnos_idx)

            list_asist_idx.append(output)
        
    # Primero lo pasamos ambos listados por un algoritmo que relacione los nombres
    # mediante la distancia de Leviesh, retornará 3 listas. Una lista con los nombres que se logra
    # asociar, las otras 2 listas son los nombres de las listas pasadas que no lograron asociarse
    
    for fun_in_matrix in [LD,comb_min_distance]:
        
        if len(list_names_asist)==0 or len(list_alumnos_idx)==0: 
            break
        
        else:
            matrix = create_matrix(list_names_asist, list_alumnos_idx, fun_in_matrix)
    
            output, list_names_asist, list_alumnos_idx = resolve_matrix_LD(matrix,list_names_asist,list_alumnos_idx, th)
        
            list_asist_idx.append(output)

    return [list_asist_idx, list_names_asist, list_alumnos_idx]


In [598]:
output_df = get_alumno_id(df,df_alumnos)

Nombre No Incluidos del Dataset Asistencia

acevedo cabezas josefa
acuña more valezca valentina
aguiar guerrero sara
alfaro becerra dyllan amaro
alfaro becerra dyllan amaro
allende gonzalez bruno
allendes orellana jose
almea torres antony
altamirano molina alondra
andrade rubio kimberly
aponte arteaga chantall
aponte bolet sofia victoria
arancibia jara loreto
araque borrego alfredo
aravena solis agustin santiago
aravena solis agustin santiago
arela yeren angeles
arenas bolarin josefa
arenas venegas sara
arevalo perez sofia isabella
arias gamboa evans
armijo mancilla anelys
arrieta leal sanet
arrieta sanchez keyler
arrieta sanchez keyler jacob
arteaga gomez alonso
avila muñoz moises
avila peralta anthony jose
aviles oses lucas
baeza pizarro agustin
bais almarza
barcelo darwin
bardales lazaro bayron
barraza monarde sofia victoria
barraza monarde sofia victoria
barraza prudencio roque
barrios chacin jose
basabe nuñez saribeth
bastias carrasco matilde
bazalar bazalar mia
beltrami romero bi

In [599]:
output_df

,name_asist,name_id,RBD,Alumno_x,name_x,usuario_id,Alumno_y,rbd,name_y
0,abarca barraza benjamin,abarca barraza benjamin alex,25613,Abarca Barraza Benjamín,abarca barraza benjamin,187707,Abarca Barraza Benjamín Alex,25613,abarca barraza benjamin alex
1,apablaza tobar antonella,apablaza tobar antonella ignacia,25613,Apablaza Tobar Antonella,apablaza tobar antonella,187737,Apablaza Tobar Antonella Ignacia,25613,apablaza tobar antonella ignacia
2,asencio sanhueza jonas,asencio sanhueza jonas andres enrique,25613,Asencio Sanhueza Jonas,asencio sanhueza jonas,187727,Asencio Sanhueza Jonás Andrés Enrique,25613,asencio sanhueza jonas andres enrique
3,bañados villablanca javier,bañados villablanca javier ignacio,25613,Bañados Villablanca Javier,bañados villablanca javier,187766,Bañados Villablanca Javier Ignacio,25613,bañados villablanca javier ignacio
4,bustamante vasquez florencia,bustamante vasquez florencia anyaliet,25613,Bustamante Vásquez Florencia,bustamante vasquez florencia,187758,Bustamante Vásquez Florencia Anyaliet,25613,bustamante vasquez florencia anyaliet
...,...,...,...,...,...,...,...,...,...
2766,perez dominguez eidan,perez dominguez eidan alfredo tomas,25247,Pérez Domínguez Eidan,perez dominguez eidan,1968772,Pérez Dominguez Eidan Alfredo Tomás,25247,perez dominguez eidan alfredo tomas
2767,tamburini tamburini luciana,tamburini tamburini luciana angelina,25247,Tamburini Tamburini Luciana,tamburini tamburini luciana,1968773,Tamburini Tamburini Luciana Angelina,25247,tamburini tamburini luciana angelina
2768,vasquez tascon edmary,vasquez tascon edmary fernanda,25247,Vasquez Tascon Edmary,vasquez tascon edmary,755147,Vasquez Tascon Edmary Fernanda,25247,vasquez tascon edmary fernanda
2769,zamorano zuñiga angel,zamorano zuñiga angel del jesus,25247,Zamorano Zúñiga Angel,zamorano zuñiga angel,1062426,Zamorano Zúñiga Angel Del Jesús,25247,zamorano zuñiga angel del jesus


In [600]:
df_all = output_df[["Alumno_x","usuario_id"]].merge(right=df,right_on="Alumno",left_on="Alumno_x", how="inner")


In [601]:
df_all = df_alumnos.merge(right=df_all,on="usuario_id", how="inner").drop(columns=["Alumno_x"])

In [602]:
len(df_all.rbd.unique())

87

In [603]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2763 entries, 0 to 2762
Columns: 290 entries, institucion_id to 11-30
dtypes: category(2), int8(275), object(3), string(4), uint16(2), uint32(3), uint8(1)
memory usage: 966.3+ KB


In [604]:
df_all.columns

Index(['institucion_id', 'rbd', 'nombre_inst', 'curso_id', 'nivel_crs',
       'nombre_crs', 'usuario_id', 'nombre', 'apellido_paterno',
       'apellido_materno',
       ...
       '11-21', '11-22', '11-23', '11-24', '11-25', '11-26', '11-27', '11-28',
       '11-29', '11-30'],
      dtype='object', length=290)

### IVE

In [605]:
df_ive = pd.read_excel("Datos No Modificar/IVE-2022 - Basica.xlsx")
df_ive

FileNotFoundError: [Errno 2] No such file or directory: 'Datos No Modificar/IVE-2022 - Basica.xlsx'

In [ ]:
df_ive[df_ive["IVE-SINAE BÁSICA 2022"]==0][["ID_RBD","DV_RBD","DS_NOM_ESTABLE","IVE-SINAE BÁSICA 2022"]]

,ID_RBD,DV_RBD,DS_NOM_ESTABLE,IVE-SINAE BÁSICA 2022
448,771,4,ESCUELA PARTICULAR EL ARRAYAN,0.0
3910,7471,3,ESCUELA RURAL PAJARITOS,0.0
3937,7559,0,ESCUELA RURAL LOS CASTAÑOS,0.0
3938,7563,9,ESCUELA RURAL COSTA RIO BLANCO,0.0
3943,7574,4,ESCUELA RURAL SAN JOSE,0.0
4121,7868,9,ESCUELA RURAL LOS PINOS DEL EMPALME,0.0
4375,8322,4,ESCUELA RURAL HUINAY,0.0
6599,22016,7,ESCUELA PARTICULAR CARRAN CHICO,0.0


In [ ]:
df_ive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7520 entries, 0 to 7519
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID_RBD                           7520 non-null   int64  
 1   DV_RBD                           7520 non-null   int64  
 2   DS_NOM_ESTABLE                   7520 non-null   object 
 3   ID_REGION_ESTABLE                7520 non-null   int64  
 4   DS_REGION_ESTABLE                7520 non-null   object 
 5   ID_PROVINCIA_ESTABLE             7520 non-null   int64  
 6   DS_PROVINCIA_ESTABLE             7520 non-null   object 
 7   ID_COMUNA_ESTABLE                7520 non-null   int64  
 8   DS_COMUNA_ESTABLE                7520 non-null   object 
 9   ID_TIPO_DEPENDENCIA              7520 non-null   int64  
 10  DS_TIPO_DEPENDENCIA              7520 non-null   object 
 11  ID_RURALIDAD                     7520 non-null   int64  
 12  DS_RURALIDAD        

# Excel Output con la unión de todos los datos

In [606]:
multi_index = []
d = 31
x = datetime.datetime(2022,3,1)
while True:
    if x.month>11:
        break
    else:
        mes = ""
        if x.day<d:
            mes = PARSE_INT_TO_MONTH[x.month-1]
        d=x.day    
        
        multi_index.append((mes,d))
        
        x+=datetime.timedelta(days=1)

In [607]:
pd.MultiIndex.from_tuples(multi_index,names=["Mes","Día"])

MultiIndex([('Marzo',  1),
            (     '',  2),
            (     '',  3),
            (     '',  4),
            (     '',  5),
            (     '',  6),
            (     '',  7),
            (     '',  8),
            (     '',  9),
            (     '', 10),
            ...
            (     '', 21),
            (     '', 22),
            (     '', 23),
            (     '', 24),
            (     '', 25),
            (     '', 26),
            (     '', 27),
            (     '', 28),
            (     '', 29),
            (     '', 30)],
           names=['Mes', 'Día'], length=275)

In [608]:
df_all.columns = pd.MultiIndex.from_tuples([(feature,"") for feature in df_all.columns[:-275]] + multi_index)

In [609]:
df_all

institucion_id    rbd                       nombre_inst curso_id  \
                                                                        
0              4049  10622  ESCUELA PARTICULAR MARVIT SCHOOL   215545   
1              4049  10622  ESCUELA PARTICULAR MARVIT SCHOOL   215545   
2              4049  10622  ESCUELA PARTICULAR MARVIT SCHOOL   215545   
3              4049  10622  ESCUELA PARTICULAR MARVIT SCHOOL   215545   
4              4049  10622  ESCUELA PARTICULAR MARVIT SCHOOL   215545   
...             ...    ...                               ...      ...   
2758             48  10588     LICEO CARDENAL ANTONIO SAMORE   215426   
2759             48  10588     LICEO CARDENAL ANTONIO SAMORE   215426   
2760             48  10588     LICEO CARDENAL ANTONIO SAMORE   215426   
2761             48  10588     LICEO CARDENAL ANTONIO SAMORE   215426   
2762             48  10588     LICEO CARDENAL ANTONIO SAMORE   215426   

     nivel_crs nombre_crs usuario_id             nombre apellido_paterno  \
                                                                           
0            4          A     969647   Isidora Agustina          Fuentes   
1            4          A     773944    Amaro Alexander           Arraño   
2            4          A     773945       Martín Elías           Guerra   
3            4          A     126189    Vicente Ignacio        Fernández   
4            4          A     126199    Joaquín Ignacio           Llanos   
...        ...        ...        ...                ...              ...   
2758         4          A     454467   Sebastián Renato             Soto   
2759         4          A     454468  Sebastián Ignacio             Vera   
2760         4          A     454469          María Paz          Salazar   
2761         4          A     454470     Matías Gabriel          Otárola   
2762         4          A     454471      Bruno Agustín          Quichiz   

     apellido_materno  ...                                
                       ... 21 22 23 24 25 26 27 28 29 30  
0               Cerda  ...  0  0  0  0  0 -1 -1  0  0  0  
1               Angla  ...  0  1  1  1  1 -1 -1  0  0  1  
2              Zúñiga  ...  1  1  1  1  1 -1 -1  1  1  1  
3             Retamal  ...  1  0  1  1  1 -1 -1  1  1  1  
4                Mora  ...  1  1  1  1  1 -1 -1  1  1  1  
...               ...  ... .. .. .. .. .. .. .. .. .. ..  
2758        Gutiérrez  ...  1  0  1  1  1 -1 -1  0  0  0  
2759       Bustamante  ...  1  1  1  1  1 -1 -1  1  1  1  
2760          Rapimán  ...  1  1  1  0  0 -1 -1  1  1  1  
2761         Galdames  ...  1  1  0  1  1 -1 -1  0  1  1  
2762           Bernal  ...  1  1  1  1  1 -1 -1  1  1  1  

[2763 rows x 290 columns]

In [610]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2763 entries, 0 to 2762
Columns: 290 entries, ('institucion_id', '') to ('', 30)
dtypes: category(2), int8(275), object(3), string(4), uint16(2), uint32(3), uint8(1)
memory usage: 966.3+ KB


In [611]:
df_all.to_excel("output.xlsx")

# Excel con los datos del gobierno

In [612]:
path_data_gob = Path("C:/Users/Sebastián Carrasco/Desktop/Practica CIAE/Automización Asisitencia/Datos del Gobierno/Asistencias/2022/")
dir_aux = [path for path in path_data_gob.glob("**/*.csv")]
#list_df = [pd.read_csv(csv, delimiter=";", encoding = "ISO-8859-1") for csv in dir_aux]

In [613]:
def filtrar_dataset_gobierno(list_path: list[Path])->pd.DataFrame:
    # Creamos una lista en donde guardamos los datasets del gobierno
    # ya filtrados
    list_df: list[pd.DataFrame] = []
    # Recorremos todos los posibles directorios dados
    for path in list_path:
        # Leemos un dataset
        df: pd.DataFrame = pd.read_csv(path, delimiter=";", encoding = "ISO-8859-1")
        # Empezamos a filtrar los datos que no nos son utiles
        

In [614]:
aux = pd.read_csv(dir_aux[0], delimiter=";", encoding = "utf-16")

UnicodeError: UTF-16 stream does not start with BOM

In [ ]:
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3404801 entries, 0 to 3404800
Data columns (total 27 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   AGNO             int64 
 1   MES_ESCOLAR      int64 
 2   RBD              int64 
 3   DGV_RBD          int64 
 4   NOM_RBD          object
 5   COD_REG_RBD      int64 
 6   NOM_REG_RBD_A    object
 7   COD_PRO_RBD      int64 
 8   COD_COM_RBD      int64 
 9   NOM_COM_RBD      object
 10  COD_DEPROV_RBD   int64 
 11  NOM_DEPROV_RBD   object
 12  RURAL_RBD        int64 
 13  COD_DEPE         int64 
 14  COD_DEPE2        int64 
 15  COD_ENSE         int64 
 16  COD_ENSE2        int64 
 17  COD_GRADO        int64 
 18  LET_CUR          object
 19  MRUN             int64 
 20  GEN_ALU          int64 
 21  FEC_NAC_ALU      int64 
 22  COD_COM_ALU      int64 
 23  NOM_COM_ALU      object
 24  DIAS_ASISTIDOS   int64 
 25  DIAS_TRABAJADOS  int64 
 26  ASIS_PROMEDIO    object
dtypes: int64(20), object(7)
memory usage: 701

In [ ]:
df_alumnos.nivel_crs.unique()

array([4], dtype=uint8)

In [ ]:
rbds = df_alumnos.rbd.unique()
nivel_crs = df_alumnos.nivel_crs.unique()[0]

aux[(aux.RBD.map( lambda x: x in rbds)) & (aux.COD_GRADO == nivel_crs)].MES_ESCOLAR.unique()

array([3], dtype=int64)